This notebook was used to created the extended OHDSI mapping

In [17]:
import sys
import os

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(os.path.join(parent_dir, "src"))

import polars as pl
from file_paths import MAPPING_DIR,  MIMIC_MEDS_DIR, OUTPUT_DIR
from mapping.metadata import load_metadata, get_number_of_mappings, save_metadata

In [19]:
mapping_schema = {
    "vocabulary_id": pl.Utf8,
    "target_concept_id": pl.Int64,
    "source_code": pl.Utf8
}

mapping_df = pl.read_csv(os.path.join(MAPPING_DIR, "mapping_results", "final_mappings.csv"), schema_overrides=mapping_schema)
mapping_df

mapping_source_table,table_name,etl_prefix,source_code,vocabulary_id,target_concept_id,ontology_concept_name
str,str,str,str,str,i64,str
"""gcpt_drug_ndc""","""prescriptions""","""NDC/""","""61314020315""","""RxNorm/373454""",40076463,"""pilocarpine Ophthalmic Solutio…"
"""gcpt_drug_ndc""","""prescriptions""","""NDC/""","""00264958720""","""RxNorm/5224""",1367571,"""heparin"""
"""gcpt_drug_ndc""","""prescriptions""","""NDC/""","""00009082501""","""RxNorm/5492""",975125,"""hydrocortisone"""
"""gcpt_drug_ndc""","""prescriptions""","""NDC/""","""00338055002""","""RxNorm/5224""",1367571,"""heparin"""
"""gcpt_drug_ndc""","""prescriptions""","""NDC/""","""00904150061""","""RxNorm/197900""",19019112,"""lorazepam 0.5 MG Oral Tablet"""
…,…,…,…,…,…,…
"""gcpt_per_ethnicity""","""admissions""","""MIMIC_IV_Race/""","""OBSERVATION ADMIT""","""CMS Place of Service/OMOP48220…",581385,"""Observation Room"""
"""gcpt_per_ethnicity""","""admissions""","""MIMIC_IV_Race/""","""SURGICAL SAME DAY ADMISSION""","""CMS Place of Service/24""",8883,"""Ambulatory Surgical Center"""
"""gcpt_per_ethnicity""","""admissions""","""MIMIC_IV_Race/""","""URGENT""","""Visit/ERIP""",262,"""Emergency Room and Inpatient V…"


In [20]:
metadata = load_metadata(MIMIC_MEDS_DIR)
code_metadata = metadata["code_metadata"]

In [21]:
get_number_of_mappings(code_metadata)

2091

In [22]:
for row in mapping_df.iter_rows(named=True):
    child_code = row["etl_prefix"] + row["source_code"]
    parent_code = row["vocabulary_id"]
    description = row["ontology_concept_name"]
    if description is not None:
        if not child_code in code_metadata:
            code_metadata[child_code] = {
                "description": description,
                "parent_codes": [parent_code]
            }
        else:
            if "parent_codes" in code_metadata[child_code]:
                if not parent_code in code_metadata[child_code]["parent_codes"]:
                    code_metadata[child_code]["parent_codes"].append(parent_code)
            else:
                code_metadata[child_code]["parent_codes"] = [parent_code]

In [23]:
get_number_of_mappings(code_metadata)

3418

In [24]:
save_metadata(os.path.join(OUTPUT_DIR, "mapping-metadata"), metadata)